In [1]:
import pickle
import numpy as np
from itertools import chain, combinations
import time
import tqdm as tq

In [2]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../../Sampler/')


from sample_a import get_Nikt, update_A, get_Ndkv
from sample_phi_joint import update_Phi
from sample_z_HB import update_Z_Hammingball as update_Z
from sample_x import update_X_exact as update_X

In [3]:
PIK = "../../Data/Syndata7N.dat"

with open(PIK, "rb") as f:
    trueW = pickle.load(f)
    trueA = pickle.load(f)
    trueX = pickle.load(f)
    trueZ = pickle.load(f)
    trueRho = pickle.load(f)
    truePhi = pickle.load(f)
    trueGam = pickle.load(f)
    Nt = pickle.load(f)
    eta1 = pickle.load(f)
    eta2 = pickle.load(f)
    alpha = pickle.load(f)
    beta = pickle.load(f)

In [4]:
D, T = trueW.shape[1], trueW.shape[2]

**Inference is made against a cross-sectional data, obtained from underlying data generating process**

The code runs the MCMC chain to infer $A, Z, \phi, \gamma,  W$. The chain for each parameter is coded in function.



Construct the MCMC chain for $A, Z, \phi, \gamma,  W$

In [5]:
D, T = trueW.shape[1], trueW.shape[2]
K = trueZ.shape[1]
print("Setting of the experiment\n")
print("Number of observations across time : {}\n".format(Nt))
print("Number of underlying clusters K: {}\n".format(K))
print("Number of diseases D: {}\n".format(D))
print("Number of time points T: {}\n".format(T))

Setting of the experiment

Number of observations across time : [450 450 450 450 450 450 450 450 450 450]

Number of underlying clusters K: 4

Number of diseases D: 20

Number of time points T: 10



Posterior inference of $X, Z, \Phi, \gamma$ simulatenously

In [6]:
print("alpha: {}".format(alpha) + " beta: {}".format(beta))
print("eta1: {}".format(eta1) + " eta2: {}".format(eta2))

alpha: 0.25 beta: 1.0
eta1: 0.3 eta2: 0.3


In [7]:
# Hamming Ball Radius
R = 2

M = 50

# True parameters
# alpha = 1/K
# beta = 1.

# eta1 = 0.3
# eta2 = 0.3

diffusion_unit = 0.8
propScale = 0.1
Times = []

In [8]:
APost = []
XPost = []
ZPost = []
PhiPost = []
NdkvPost = []

1. Load the initial parameters.

$X, Z, A$ : Assign random values within its support

$\phi, \gamma$ : Draw from their prior distribution.

In [9]:
PIK = "init.dat"

with open(PIK, "rb") as f:
    initA = pickle.load(f)
    initZ = pickle.load(f)
    initX = pickle.load(f)
    initPhi = pickle.load(f)

In [10]:
APost.append(initA)
ZPost.append(initZ)
XPost.append(initX)
PhiPost.append(initPhi)

In [11]:
Count = 0

2. Run MCMC algorithm for 10,000 iterations.

In [12]:
# Original
np.random.seed(15)
IT = 10000


for i in tq.tqdm(range(1,IT+1)):
    begin = time.time()
    if i == 1:
        Ndkv = get_Ndkv(APost[i-1], trueW, Nt, K)
        Nikt = get_Nikt(APost[i-1], K, Nt)
    else:
        APost.append(update_A(APost[i-2], trueW, Ndkv, Nikt, ZPost[i-1], PhiPost[i-1], Nt, eta1, eta2))
        Ndkv = get_Ndkv(APost[i-1], trueW, Nt, K)
        Nikt = get_Nikt(APost[i-1], K, Nt)
        
    ZPost.append(update_Z(XPost[i-1], PhiPost[i-1], Nikt, Nt, R))
        
    phi, Count = update_Phi(PhiPost[i-1], ZPost[i], Nikt, D, Count, trueGam, propScale)
    PhiPost.append(phi)
    
    XPost.append(update_X(XPost[i-1], ZPost[i], Nt, M, alpha, beta, diffusion_unit))
    end = time.time()
    Times.append(end-begin)
    
    NdkvPost.append(Ndkv)
    
    if i in np.linspace(100,IT,100).astype(int):
        if i == 100:
            with open('../../MCMC/UM03/sample_'+str(i)+'.dat', "wb") as f:
                pickle.dump(XPost[-101:], f)
                pickle.dump(ZPost[-101:], f)
                pickle.dump(PhiPost[-101:], f)
                pickle.dump(NdkvPost[-101:], f)
                pickle.dump(APost[-101:], f)
                pickle.dump(Count, f)
        else:
            with open('../../MCMC/UM03/sample_'+str(i)+'.dat', "wb") as f:
                pickle.dump(XPost[-100:], f)
                pickle.dump(ZPost[-100:], f)
                pickle.dump(PhiPost[-100:], f)
                pickle.dump(NdkvPost[-100:], f)
                pickle.dump(APost[-100:], f)
                pickle.dump(Count, f)

100%|████████████████████████████████████| 10000/10000 [23:09:29<00:00,  8.34s/it]


In [13]:
with open('../../MCMC/UM03/timespent.dat', "wb") as f:
    pickle.dump(Times, f)